In [1]:
import os
import tensorflow as tf

In [ ]:
model_method_II = tf.keras.models.load_model('//153.19.52.107/emboa/IO3-sessions/NEW STRUCTURE/de-earlyfusionthesis/Datasets/saved_model/model.keras')

In [ ]:
model_method_II.summary()

In [ ]:
path = r'S:\IO3-sessions\NEW STRUCTURE\de-earlyfusionthesis\Datasets'

train_dataset_path = os.path.abspath(os.path.join(path, 'train_dataset'))
GUT_test_path = os.path.join(path, 'GUT_test').replace("\\", "/")
ITU_YU_test_path = os.path.join(path, 'ITU_YU_test').replace("\\", "/")
MAAP_test_path = os.path.join(path, 'MAAP_test').replace("\\", "/")

# Load datasets
train = tf.data.Dataset.load(train_dataset_path)
GUT_test = tf.data.Dataset.load(GUT_test_path)
ITU_YU_test = tf.data.Dataset.load(ITU_YU_test_path)
MAAP_test = tf.data.Dataset.load(MAAP_test_path)

## Model I

For the first model we can use in-bulit functions of False positives and so on...

## Model II

In [ ]:
import tensorflow as tf
test_set = GUT_test
# Getting predictions (chat)
y_pred = model_method_II.predict(test_set.map(lambda x, _: x))
y_true = tf.concat([y for _, y in test_set], axis=0)

# Function
def compute_similarity(y_true, y_pred, emotion_index):
    # Ensure tensors are of the same type cause some bugs idk
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    
    # Extract true and predicted values for the specific emotion
    true_values = y_true[:, :, emotion_index]
    pred_values = y_pred[:, :, emotion_index]
    
    # Create a mask for non-zero true values (bylo na wloskich zajeciach)
    mask = tf.not_equal(true_values, 0)
    
    # Apply the mask
    true_values_masked = tf.boolean_mask(true_values, mask)
    pred_values_masked = tf.boolean_mask(pred_values, mask)
    
    # Compute the similarity: 1 - |true - prediction|
    similarity = 1 - tf.abs(true_values_masked - pred_values_masked)

    # Compute Mean Average Similarity (MAS)
    mas = tf.reduce_mean(similarity)
    
    # How many labels were used for this metric
    label_count = tf.size(true_values_masked).numpy()
    
    return mas.numpy(), label_count

# List for emotions
mas_per_emotion = []
label_counts = []
emotions = ["Happy", "Sad", "Angry", "Fear", "Disgusted", "Surprised"]

for i, emotion in enumerate(emotions):
    mas, label_count = compute_similarity(y_true, y_pred, i)
    mas_per_emotion.append(mas)
    label_counts.append(label_count)
    print(f"Similarity for {emotion}: {mas}, Labels Used = {label_count}")